In [24]:
import os
import re
import requests
import json
import pandas as pd

class SpellChecker:
    def __init__(self):
        self.passport_key = None
        self.base_url = None

    def chunk_text_by_words(self, text, max_length=500):
        words = text.split()
        chunks = []
        current_chunk = []
        for word in words:
            if len(' '.join(current_chunk + [word])) <= max_length:
                current_chunk.append(word)
            else:
                chunks.append(' '.join(current_chunk))
                current_chunk = [word]
        if current_chunk:
            chunks.append(' '.join(current_chunk))
        return chunks

    def fetch_passport_key(self):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
            'Referer': 'https://search.naver.com/',
        }
        response = requests.get("https://search.naver.com/search.naver?query=%EB%A7%9E%EC%B6%A9%EB%B2%95%20%EA%B2%80%EC%82%AC%EA%B8%B0", headers=headers)
        passport_key_match = re.search(r'(?<=checker:").*?(?="},selector)', response.text)
        if not passport_key_match:
            return "Error: Unable to retrieve passport key"
        api_url = passport_key_match.group(0)
        self.base_url, self.passport_key = api_url.split("?passportKey=")

    def spell_check(self, text):
        if self.passport_key is None or self.base_url is None:
            self.fetch_passport_key()
        chunks = self.chunk_text_by_words(text, 500)
        checked_text = []
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/129.0.0.0 Safari/537.36',
            'Referer': 'https://search.naver.com/',
        }
        for chunk in chunks:
            payload = {
                'passportKey': self.passport_key,
                'where': 'nexearch',
                'color_blindness': 0,
                'q': chunk
            }
            result_response = requests.get(self.base_url, headers=headers, params=payload)
            result = json.loads(result_response.text)
            corrected_chunk = result['message']['result']['notag_html']
            checked_text.append(corrected_chunk)
        return ' '.join(checked_text)

checker = SpellChecker()

In [25]:
file_name = "result.csv"
path_to_in = os.path.join("/home/chio4696/DLthon_3team/DATA/AI_Hub/", file_name)
path_to_out = os.path.join("/home/chio4696/DLthon_3team/DATA/AI_Hub/", "new_"+file_name)

def load_text(path_to_in, path_to_out):
    df = pd.read_csv(path_to_in)
    print(df)
    df['conversation'] = df['conversation'].astype(str).apply(checker.spell_check)
    df.to_csv(path_to_out, header=False)
    return df
load_text(path_to_in, path_to_out)

       idx      class                                       conversation
0        0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1        1      협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2        2  기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3        3      갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4        4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...    ...        ...                                                ...
4945  4945      일반 대화  한달살기로 가보고 싶은 지역이 있어?\n 우리나라 중에서 말하는거야?\n 국내, 해...
4946  4946      일반 대화  그런 스트레스는 키키 재밌는 드라마나 영화나 예능이 최고지!\n 난 요즘 갯마을 차...
4947  4947      일반 대화  나 저번에 소개팅했던 남자한테 연락이 왔어\n 그래? 뭐라고 왔니?\n 궁금하다 저...
4948  4948      일반 대화  우리나라 자국산 전투기 만드는거 알아?\n 전투기를 왜 만드는거야?\n 나라를 지키...
4949  4949      일반 대화  그래도 출퇴근이라도 편하면 좋지 키키\n 뭐... 그게 나쁜 건 아니지...\n 회...

[4950 rows x 3 columns]


,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼...
1,1,협박 대화,길동 경찰서입니다. 9시 40분 마트에 폭발물을 설치할 거다. 네? 똑바로 들어 한...
2,2,기타 괴롭힘 대화,너 되게 귀여운 거 알지? 나보다 작은 남자는 첨 봤어. 그만해. 너희들 놀리는 거...
3,3,갈취 대화,어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아 보인다? 얘 ...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
4945,4945,일반 대화,한 달 살기로 가보고 싶은 지역이 있어?\n 우리나라 중에서 말하는 거야?\n 국내...
4946,4946,일반 대화,그런 스트레스는 키키 재밌는 드라마나 영화나 예능이 최고지!\n 난 요즘 갯마을 차...
4947,4947,일반 대화,나 저번에 소개팅했던 남자한테 연락이 왔어\n 그래? 뭐라고 왔니?\n 궁금하다 저...
4948,4948,일반 대화,우리나라 자국산 전투기 만드는 거 알아?\n 전투기를 왜 만드는 거야?\n 나라를 ...
